训练zinc数据集的指令为
`python train_zinc.py --abs-pe rw --se khopgnn --gnn-type pna2 --dropout 0.2 --k-hop 3 --use-edge-attr`

In [ ]:

'''
Namespace(abs_pe='rw',
            abs_pe_dim=20,
            batch_norm=True,
            batch_size=128,
            crystal_dataset='ZINC',
            dim_hidden=64, dropout=0.2,
            edge_dim=32,
            epochs=2000,
            global_pool='mean',
            gnn_type='pna2',
            k_hop=3,
            layer_norm=False,
            lr=0.001,
            num_heads=8,
            num_layers=6,
            outdir='',
            save_logs=False,
            se='khopgnn',
            seed=0,
            use_cuda=True,
            use_edge_attr=True,
            warmup=5000,
            weight_decay=1e-05)
'''

在这个project中，train_zinc是可以运行的，请放心，接下来，我们来对比看看zinc数据和crystal数据的差别，为什么crystal经过转化之后就不可以运行

# import here

In [133]:
import importlib
from csat.data import GraphDataset

from torch_geometric.data import DataLoader
from torch_geometric import datasets
import csat.crytal_data
importlib.reload(csat.crytal_data)
from csat.crytal_data import crystal_graph_list,CIFData
from csat.models import GraphTransformer
import os
import copy
import argparse
import numpy as np
import pandas as pd
from collections import defaultdict
import torch
from torch import nn, optim
import torch.nn.functional as F
import torch_geometric.utils as utils
from torch.utils.data import  Subset

import warnings
warnings.filterwarnings("ignore")

abs_pe='rw'
k_hop = 2
se = 'khopgnn'
use_edge_attr = True
abs_pe='rw'
abs_pe_dim=20
batch_norm=True
batch_size=128
dim_hidden=64
dropout=0.2
edge_dim=32
epochs=50
global_pool='mean'
gnn_type='pna2'
layer_norm=False
lr=0.001
num_heads=8
num_layers=6
outdir=''
save_logs=False
seed=0
use_cuda=True
epochs = 50
warmup=5000
weight_decay=1e-05
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 文件地址

In [123]:
zinc_data_path = './zinc_datasets/ZINC'
crystal_data_path = './sample-regression'

# ZINC and Crystal

In [152]:
train_zinc = datasets.ZINC(zinc_data_path, subset=True,split='train')
print(len(train_zinc))
#indices = range(0, 10) # 1~10
#train_zinc = torch.utils.data.Subset(train_zinc, indices)



print(train_zinc,'zinc数据集是一个图的list集合。')
print(type(train_zinc))
print('zinc数据集图的数量：',len(train_zinc))
print('zinc数据集第1张图的数据类型为：',train_zinc[0])
print('zinc数据集第10张图的数据类型为：',train_zinc[9])

10000
ZINC(10000) zinc数据集是一个图的list集合。
<class 'torch_geometric.datasets.zinc.ZINC'>
zinc数据集图的数量： 10000
zinc数据集第1张图的数据类型为： Data(x=[29, 1], edge_index=[2, 64], edge_attr=[64], y=[1])
zinc数据集第10张图的数据类型为： Data(x=[18, 1], edge_index=[2, 38], edge_attr=[38], y=[1])


在这里有一个矛盾的地方：

官方说明zinc数据集中num_node_fea应该是28，这是全部为1，启发了我的降维想法

In [124]:
crystal_data = CIFData(crystal_data_path)
print(crystal_data)
print("crystal data 是一个元组的形式，其数据结构为(atom_fea, nbr_fea, nbr_fea_idx), target, cif_id")
print("对应的图数据类型为                             x,  edge_attr,   edge_index")
c_structures, target, cif_id = crystal_data[0]
print('atom_fea: (n_i, atom_fea_len)',c_structures[0].shape)
print('nbr_fea: (n_i, M, nbr_fea_len)',c_structures[1].shape)
print('nbr_fea_idx: (n_i, M)',c_structures[2].shape)
print("于是，我们需要把他转化为与zinc一样的graph_list")

crystal data 是一个元组的形式，其数据结构为(atom_fea, nbr_fea, nbr_fea_idx), target, cif_id
对应的图数据类型为                             x,  edge_attr,   edge_index
atom_fea: (n_i, atom_fea_len) torch.Size([8, 92])
nbr_fea: (n_i, M, nbr_fea_len) torch.Size([8, 12, 41])
nbr_fea_idx: (n_i, M) torch.Size([8, 12])
于是，我们需要把他转化为与zinc一样的graph_list


## 从此处隔断

In [12]:
crystal_list = crystal_graph_list(crystal_data)
print('这是crystal list的数据类型',type(crystal_list))
print('crystal list图的数量：',len(crystal_list))
print('crystal list第1张图的数据类型为',crystal_list[0])
print('crystal list第10张图的数据类型为',crystal_list[9])

crystal data 是一个元组的形式，其数据结构为(atom_fea, nbr_fea, nbr_fea_idx), target, cif_id
对应的图数据类型为                             x,       
于是，我们需要把他转化为与zinc一样的graph_list
正在转化的数据集大小为： 10


100%|██████████| 10/10 [00:00<00:00, 43.00it/s]

这是crystal list的数据类型 <class 'list'>
crystal list图的数量： 10
crystal list第1张图的数据类型为 Data(x=[8, 92], edge_index=[8, 12], edge_attr=[8, 12, 41], y=[1])
crystal list第10张图的数据类型为 Data(x=[8, 92], edge_index=[8, 12], edge_attr=[8, 12, 41], y=[1])


由此我们可以发现

zinc：


zinc数据集第1张图的数据类型为： Data(x=[29, 1], edge_index=[2, 64], edge_attr=[64], y=[1])

zinc数据集第100张图的数据类型为： Data(x=[27, 1], edge_index=[2, 60], edge_attr=[60], y=[1])

crystal：

crystal list第1张图的数据类型为 Data(x=[8, 92], edge_index=[8, 12], edge_attr=[8, 12, 41], y=[1])

crystal list第10张图的数据类型为 Data(x=[8, 92], edge_index=[8, 12], edge_attr=[8, 12, 41],y=[1])

在CIFDATA数据中
```text
atom_fea: torch.Tensor shape (n_i, atom_fea_len)
nbr_fea: torch.Tensor shape (n_i, M, nbr_fea_len)
nbr_fea_idx: torch.LongTensor shape (n_i, M)
target: torch.Tensor shape (1, )
```

我们就需要对其做出修改,将egde_index改为[2, num_edges],将edge_attr降维为1维度。

In [26]:
print('这是重新编写之后的crystal list！')
crystal_list = crystal_graph_list(crystal_data)
print('这是crystal list的数据类型',type(crystal_list))
print('crystal list图的数量：',len(crystal_list))
print('crystal list第1张图的数据类型为',crystal_list[0])
print('crystal list第10张图的数据类型为',crystal_list[9])

这是重新编写之后的crystal list！
正在转化的数据集大小为： 10


100%|██████████| 10/10 [00:00<00:00, 5024.32it/s]

这是crystal list的数据类型 <class 'list'>
crystal list图的数量： 10
crystal list第1张图的数据类型为 Data(x=[8, 92], edge_index=[2, 48], edge_attr=[96], y=[1])
crystal list第10张图的数据类型为 Data(x=[8, 92], edge_index=[2, 48], edge_attr=[96], y=[1])


这是
然后调用sat的graphdataset提取子图

In [97]:
train_zinc_dset = GraphDataset(train_zinc,degree=True, k_hop=k_hop, se=se,use_subgraph_edge_attr=use_edge_attr)
print(train_zinc_dset)
print('含有子图第一张图数据为：',train_zinc_dset[0])

Extracting 2-hop subgraphs...
Done!
含有子图第一张图数据为： Data(x=[29], edge_index=[2, 64], edge_attr=[64], y=[1, 1], complete_edge_index=[2, 841], degree=[29], subgraph_edge_index=[2, 326], num_subgraph_nodes=187, subgraph_node_idx=[187], subgraph_edge_attr=[326], subgraph_indicator=[187])


In [163]:
train_c_dset = GraphDataset(crystal_list,degree=True, k_hop=k_hop, se=se,use_subgraph_edge_attr=use_edge_attr)
print('含有子图的crystal数据集第一张图数据为：',trian_c_dset[0])

Extracting 2-hop subgraphs...
Done!
含有子图的crystal数据集第一张图数据为： Data(x=[8, 92], edge_index=[2, 12], edge_attr=[12], y=[1, 1], complete_edge_index=[2, 64], subgraph_edge_index=[2, 14], num_subgraph_nodes=15, subgraph_node_idx=[15], subgraph_edge_attr=[14], subgraph_indicator=[15], degree=[8])


我的原数据为`Data(x=[8, 92], edge_index=[8, 12], edge_attr=[8, 12, 41], y=[1])`，再经过edge_index转化[2,n]以及edge_attr降维之后变成：`Data(x=[8, 92], edge_index=[2, 48], edge_attr=[96], y=[1])`。

于是出现了问题：edge_mask 是一个布尔张量，用于选择与当前子图相关的边。它的长度应该等于graph.edge_index 的边数（即 graph.edge_index.shape[1]）。但是graph.edge_attr 的长度是 96，因此无法用 edge_mask 对 graph.edge_attr 进行索引。

因此对crystal_graph_list进行改写，使用了稀疏矩阵的方法重新改写，再测试

## 从此处开始运行

In [125]:
print('这是第二次重新编写之后的crystal list！')
crystal_list = crystal_graph_list(crystal_data)
print('这是crystal list的数据类型',type(crystal_list))
print('crystal list图的数量：',len(crystal_list))
print('crystal list第1张图的数据类型为',crystal_list[0])
print('crystal list第10张图的数据类型为',crystal_list[9])

这是第二次重新编写之后的crystal list！
正在转化的数据集大小为： 10


100%|██████████| 10/10 [00:00<00:00, 55.48it/s]

这是crystal list的数据类型 <class 'list'>
crystal list图的数量： 10
crystal list第1张图的数据类型为 Data(x=[8, 1], edge_index=[2, 96], edge_attr=[96, 41], y=[1])
crystal list第10张图的数据类型为 Data(x=[8, 1], edge_index=[2, 96], edge_attr=[96, 41], y=[1])


结果表明很成功

In [126]:
train_c_dset = GraphDataset(crystal_list,degree=True, k_hop=k_hop, se=se,use_subgraph_edge_attr=use_edge_attr)
print('c',train_c_dset[0])

train_zinc_dset = GraphDataset(train_zinc, degree=True, k_hop=k_hop, se=se, use_subgraph_edge_attr=use_edge_attr)
print(train_zinc_dset)
print('z', train_zinc_dset[0])

Extracting 2-hop subgraphs...
Done!
c Data(x=[8], edge_index=[2, 96], edge_attr=[96, 41], y=[1, 1], complete_edge_index=[2, 64], degree=[8], subgraph_edge_index=[2, 768], num_subgraph_nodes=64, subgraph_node_idx=[64], subgraph_edge_attr=[768, 41], subgraph_indicator=[64])
Extracting 2-hop subgraphs...
Done!
z Data(x=[29], edge_index=[2, 64], edge_attr=[64], y=[1, 1], complete_edge_index=[2, 841], degree=[29], subgraph_edge_index=[2, 326], num_subgraph_nodes=187, subgraph_node_idx=[187], subgraph_edge_attr=[326], subgraph_indicator=[187])


# 检查model(dataset)
在开始训练之前，出现了如下：

RuntimeError: The expanded size of the tensor (276) must match the existing size (3) at non-singleton dimension 0.  Target sizes: [276, 64].  Tensor sizes: [3, 1]

In [85]:
print('zinc数据集第1张图的数据类型为：',train_zinc[0])
print('\n')
print('含有子图的zinc第1张',train_zinc_dset[0])
print('\n')
print('zinc数据集在loader前',train_zinc_dset[0])
train_zinc_loader = DataLoader(train_zinc_dset, batch_size=128,shuffle=True)
print('\n')
for batch in train_zinc_loader:
    print('loader之后：',batch)

zinc数据集第1张图的数据类型为： Data(x=[29, 1], edge_index=[2, 64], edge_attr=[64], y=[1])


含有子图的zinc第1张 Data(x=[29], edge_index=[2, 64], edge_attr=[64], y=[1, 1], complete_edge_index=[2, 841], degree=[29], subgraph_edge_index=[2, 326], num_subgraph_nodes=187, subgraph_node_idx=[187], subgraph_edge_attr=[326], subgraph_indicator=[187])


zinc数据集在loader前 Data(x=[29], edge_index=[2, 64], edge_attr=[64], y=[1, 1], complete_edge_index=[2, 841], degree=[29], subgraph_edge_index=[2, 326], num_subgraph_nodes=187, subgraph_node_idx=[187], subgraph_edge_attr=[326], subgraph_indicator=[187])


loader之后： DataBatch(x=[222], edge_index=[2, 472], edge_attr=[472], y=[10, 1], complete_edge_index=[2, 5152], degree=[222], subgraph_edge_index=[2, 2310], num_subgraph_nodes=[10], subgraph_node_idx=[1334], subgraph_edge_attr=[2310], subgraph_indicator=[1334], batch=[222], ptr=[11])


E:\anaconda3\envs\pytorch\lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


请注意！由于数据量过小，在loader之后，仅有一个

In [127]:
print('c数据集第1张图的数据类型为：',crystal_list[0])
print('\n')
print('含有子图的c第1张',train_c_dset[0])
print('\n')
print('crystal数据集在loader前',train_c_dset[0])
print('\n')

train_c_loader = DataLoader(train_c_dset, batch_size=128, shuffle=True)
for batch in train_c_loader:
    print('loader之后：',batch)
    break

c数据集第1张图的数据类型为： Data(x=[8], edge_index=[2, 96], edge_attr=[96, 41], y=[1, 1], complete_edge_index=[2, 64], degree=[8], subgraph_edge_index=[2, 768], num_subgraph_nodes=64, subgraph_node_idx=[64], subgraph_edge_attr=[768, 41], subgraph_indicator=[64])


含有子图的c第1张 Data(x=[8], edge_index=[2, 96], edge_attr=[96, 41], y=[1, 1], complete_edge_index=[2, 64], subgraph_edge_index=[2, 768], num_subgraph_nodes=64, subgraph_node_idx=[64], subgraph_edge_attr=[768, 41], subgraph_indicator=[64], degree=[8])


crystal数据集在loader前 Data(x=[8], edge_index=[2, 96], edge_attr=[96, 41], y=[1, 1], complete_edge_index=[2, 64], subgraph_edge_index=[2, 768], num_subgraph_nodes=64, subgraph_node_idx=[64], subgraph_edge_attr=[768, 41], subgraph_indicator=[64], degree=[8])


loader之后： DataBatch(x=[65], edge_index=[2, 780], edge_attr=[780, 41], y=[10, 1], complete_edge_index=[2, 513], degree=[65], subgraph_edge_index=[2, 5772], num_subgraph_nodes=[10], subgraph_node_idx=[481], subgraph_edge_attr=[5772, 41], subgraph

从数据上看，没有什么问题

#### zincmox

In [70]:
deg = torch.cat([
        utils.degree(data.edge_index[1], num_nodes=data.num_nodes) for
        data in train_zinc_dset])
zinc_model = GraphTransformer(in_size=28,
                             num_class=1,
                             d_model=dim_hidden,
                             dim_feedforward=2 * dim_hidden,
                             dropout=dropout,
                             num_heads=num_heads,
                             num_layers=num_layers,
                             batch_norm=batch_norm,
                             abs_pe=abs_pe,
                             abs_pe_dim=abs_pe_dim,
                             gnn_type=gnn_type,
                             use_edge_attr=use_edge_attr,
                             num_edge_features=4,
                             edge_dim=edge_dim,
                             k_hop=k_hop,
                             se=se,
                             deg=deg,
                             global_pool=global_pool)

在这里需要注意一个问题

zinc数据集中

number of node attributes for ZINC crystal_dataset

n_tags = 28------------>in_size

num_edge_features = 4

而我们crystal数据集的这两个参数是未知的，于是我使用92，41代替


#### C模型

In [128]:
deg = torch.cat([
        utils.degree(data.edge_index[1], num_nodes=data.num_nodes) for
        data in train_c_dset])
c_model = GraphTransformer(in_size=92,
                             num_class=1,
                             d_model=dim_hidden,
                             dim_feedforward=2 * dim_hidden,
                             dropout=dropout,
                             num_heads=num_heads,
                             num_layers=num_layers,
                             batch_norm=batch_norm,
                             abs_pe=abs_pe,
                             abs_pe_dim=abs_pe_dim,
                             gnn_type=gnn_type,
                             use_edge_attr=use_edge_attr,
                             num_edge_features=41,
                             edge_dim=edge_dim,
                             k_hop=k_hop,
                             se=se,
                             deg=deg,
                             global_pool=global_pool)

#### 训练函数

In [120]:
def train_out(loader, model, optimizer):
    running_loss = 0.0
    model.train()
    for i, data in enumerate(loader):
        print(i)
        print(data)
        size = len(data.y)
        if use_cuda:
            data = data.cuda()

        criterion = nn.L1Loss()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data.y)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * size

    n_sample = len(loader.dataset)
    epoch_loss = running_loss / n_sample

    print('done!')
    print('epoch_loss is',epoch_loss)
    return epoch_loss

In [89]:
zinc_model = zinc_model.to(device)
optimizer = optim.AdamW(zinc_model.parameters(), lr=lr, weight_decay=weight_decay)
zinc_out = train_out(train_zinc_loader, zinc_model,optimizer)


0
DataBatch(x=[222], edge_index=[2, 472], edge_attr=[472], y=[10, 1], complete_edge_index=[2, 5152], degree=[222], subgraph_edge_index=[2, 2310], num_subgraph_nodes=[10], subgraph_node_idx=[1334], subgraph_edge_attr=[2310], subgraph_indicator=[1334], batch=[222], ptr=[11])
done!
epoch_loss is 0.9110336303710938


In [129]:
c_model = c_model.to(device)
optimizer = optim.AdamW(c_model.parameters(), lr=lr, weight_decay=weight_decay)
c_out = train_out(train_c_loader, c_model,optimizer)

0
DataBatch(x=[65], edge_index=[2, 780], edge_attr=[780, 41], y=[10, 1], complete_edge_index=[2, 513], subgraph_edge_index=[2, 5772], num_subgraph_nodes=[10], subgraph_node_idx=[481], subgraph_edge_attr=[5772, 41], subgraph_indicator=[481], degree=[65], batch=[65], ptr=[11])


RuntimeError: Sizes of tensors must match except in dimension 2. Expected size 5772 but got size 236652 for tensor number 2 in the list.

# 现在对5000si数据进行同样的测试

In [102]:
crystal_data_path_5000 = './crystal_dataset'
crystal_data_5000 = CIFData(crystal_data_path_5000)
crystal_list = crystal_graph_list(crystal_data_5000)
print('这是crystal list 5000的数据类型',type(crystal_list))
print('crystal list 5000图的数量：',len(crystal_list))
print('crystal list 5000第1张图的数据类型为',crystal_list[0])
print('crystal list 5000第10张图的数据类型为',crystal_list[9])
train_c_dset = GraphDataset(crystal_list,degree=True, k_hop=k_hop, se=se,use_subgraph_edge_attr=use_edge_attr)
print('c5000含有子图的第一张',train_c_dset[0])
train_c_loader = DataLoader(train_c_dset, batch_size=128, shuffle=True)
for batch in train_c_loader:
    print('loader之后：',batch)
    break

deg = torch.cat([
        utils.degree(data.edge_index[1], num_nodes=data.num_nodes) for
        data in train_c_dset])
c_model_5000 = GraphTransformer(in_size=92,
                             num_class=1,
                             d_model=dim_hidden,
                             dim_feedforward=2 * dim_hidden,
                             dropout=dropout,
                             num_heads=num_heads,
                             num_layers=num_layers,
                             batch_norm=batch_norm,
                             abs_pe=abs_pe,
                             abs_pe_dim=abs_pe_dim,
                             gnn_type=gnn_type,
                             use_edge_attr=use_edge_attr,
                             num_edge_features=41,
                             edge_dim=edge_dim,
                             k_hop=k_hop,
                             se=se,
                             deg=deg,
                             global_pool=global_pool)

正在转化的数据集大小为： 5000


100%|██████████| 5000/5000 [05:11<00:00, 16.04it/s]


这是crystal list 5000的数据类型 <class 'list'>
crystal list 5000图的数量： 5000
crystal list 5000第1张图的数据类型为 Data(x=[16, 1], edge_index=[2, 12], edge_attr=[12], y=[1])
crystal list 5000第10张图的数据类型为 Data(x=[80, 1], edge_index=[2, 12], edge_attr=[12], y=[1])
Extracting 2-hop subgraphs...
Done!
c5000含有子图的第一张 Data(x=[16], edge_index=[2, 12], edge_attr=[12], y=[1, 1], complete_edge_index=[2, 256], degree=[16], subgraph_edge_index=[2, 12], num_subgraph_nodes=20, subgraph_node_idx=[20], subgraph_edge_attr=[12], subgraph_indicator=[20])
loader之后： DataBatch(x=[3620], edge_index=[2, 1536], edge_attr=[1536], y=[128, 1], complete_edge_index=[2, 296368], degree=[3620], subgraph_edge_index=[2, 1661], num_subgraph_nodes=[128], subgraph_node_idx=[4282], subgraph_edge_attr=[1661], subgraph_indicator=[4282], batch=[3620], ptr=[129])


In [103]:
c_model_5000 = c_model_5000.to(device)
optimizer = optim.AdamW(c_model_5000.parameters(), lr=lr, weight_decay=weight_decay)
c_out = train_out(train_c_loader, c_model_5000,optimizer)

0
DataBatch(x=[3609], edge_index=[2, 1536], edge_attr=[1536], y=[128, 1], complete_edge_index=[2, 212887], subgraph_edge_index=[2, 1659], num_subgraph_nodes=[128], subgraph_node_idx=[4242], subgraph_edge_attr=[1659], subgraph_indicator=[4242], degree=[3609], batch=[3609], ptr=[129])
1
DataBatch(x=[3817], edge_index=[2, 1536], edge_attr=[1536], y=[128, 1], complete_edge_index=[2, 241015], subgraph_edge_index=[2, 1673], num_subgraph_nodes=[128], subgraph_node_idx=[4440], subgraph_edge_attr=[1673], subgraph_indicator=[4440], degree=[3817], batch=[3817], ptr=[129])
2
DataBatch(x=[3138], edge_index=[2, 1536], edge_attr=[1536], y=[128, 1], complete_edge_index=[2, 179014], subgraph_edge_index=[2, 1671], num_subgraph_nodes=[128], subgraph_node_idx=[3776], subgraph_edge_attr=[1671], subgraph_indicator=[3776], degree=[3138], batch=[3138], ptr=[129])
3
DataBatch(x=[3287], edge_index=[2, 1536], edge_attr=[1536], y=[128, 1], complete_edge_index=[2, 173785], subgraph_edge_index=[2, 1690], num_subgra

IndexError: Dimension specified as 0 but tensor has no dimensions

问题来了，如果把x降维成1维，5000数据集会出现IndexError: Dimension specified as 0 but tensor has no dimensions；而10个的sample数据集就没问题

如果不对x进行降维，那么cuda索引就会报错
```
RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
```

现在全部拿cpu跑，做如下统计：
1. sample-regression
    1. 不降维的报错：
        ```
        IndexError: Found indices in 'edge_index' that are larger than 91 (got 272). Please ensure that all indices in 'edge_index' point to valid indices in the interval [0, 92) in your node feature matrix and try again.
        ```
    2. 降维无问题：
        ```
        Train loss: 4.3618 time: 0.11s
        Val loss: 2.6525 MSE loss: 7.0357 MAE loss: 2.6525 time: 0.03s
        Test loss: 1.6131 MSE loss: 2.6022 MAE loss: 1.6131 time: 0.03s
        best epoch: 498 best val loss: 2.6234
        Testing...
        Test loss: 1.6366 MSE loss: 2.6785 MAE loss: 1.6366 time: 0.03s
        test MAE loss 1.6366
        ```
2. 5000 si数据
    1. x不降维度报错：
        ```
        IndexError: Found indices in 'edge_index' that are larger than 91 (got 95941). Please ensure that all indices in 'edge_index' point to valid indices in the interval [0, 92) in your node feature matrix and try again.
        ```
    2. x降至1维度
        ```
        Traceback (most recent call last):
        File "E:\WYX_Project\SAT-CGCN-Demo-1\CSAT_train_si.py", line 359, in <module>
        main()
        File "E:\WYX_Project\SAT-CGCN-Demo-1\CSAT_train_si.py", line 313, in main
        train_loss = train_epoch(model, train_loader, criterion, optimizer, lr_scheduler, epoch, args.use_cuda)
        File "E:\WYX_Project\SAT-CGCN-Demo-1\CSAT_train_si.py", line 138, in train_epoch
                for i, data in enumerate(loader):
        File "E:\anaconda3\envs\pytorch\lib\site-packages\torch\utils\data\dataloader.py", line 630, in __next__
        data = self._next_data()
        File "E:\anaconda3\envs\pytorch\lib\site-packages\torch\utils\data\dataloader.py", line 673, in _next_data
                data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
        File "E:\anaconda3\envs\pytorch\lib\site-packages\torch\utils\data\_utils\fetch.py", line 52, in fetch
                data = [self.dataset[idx] for idx in possibly_batched_index]
        File "E:\anaconda3\envs\pytorch\lib\site-packages\torch\utils\data\_utils\fetch.py", line 52, in <listcomp>
                data = [self.dataset[idx] for idx in possibly_batched_index]
        File "E:\WYX_Project\SAT-CGCN-Demo-1\csat\data.py", line 128, in __getitem__
                n = data.num_nodes
        File "E:\anaconda3\envs\pytorch\lib\site-packages\torch_geometric\data\data.py", line 616, in num_nodes
                return super().num_nodes
        File "E:\anaconda3\envs\pytorch\lib\site-packages\torch_geometric\data\data.py", line 186, in num_nodes
                return sum([v.num_nodes for v in self.node_stores])
        File "E:\anaconda3\envs\pytorch\lib\site-packages\torch_geometric\data\data.py", line 186, in <listcomp>
                return sum([v.num_nodes for v in self.node_stores])
        File "E:\anaconda3\envs\pytorch\lib\site-packages\torch_geometric\data\storage.py", line 428, in num_nodes
                return value.size(cat_dim)
            IndexError: Dimension specified as 0 but tensor has no dimensions
        ```

In [151]:
full_dataset = crystal_graph_list(crystal_data)
indices = list(range(100))
subset = Subset(full_dataset, indices)
print(full_dataset[0])
print(subset[0])

100%|██████████| 10/10 [00:00<00:00, 669.51it/s]

Data(x=[8, 92], edge_index=[2, 56], edge_attr=[56, 41], y=[1])
Data(x=[8, 92], edge_index=[2, 56], edge_attr=[56, 41], y=[1])


In [149]:
subset_graph = GraphDataset(subset, degree=True, k_hop=k_hop, se=se, use_subgraph_edge_attr=use_edge_attr)
print(subset_graph)

Extracting 2-hop subgraphs...
Done!


In [150]:
subset_loader = DataLoader(subset_graph, batch_size=128, shuffle=True)
for batch in subset_loader:
    print('loader之后：',batch)
    break

deg = torch.cat([
        utils.degree(data.edge_index[1], num_nodes=data.num_nodes) for
        data in subset_graph])
sub_model = GraphTransformer(in_size=92,
                             num_class=1,
                             d_model=dim_hidden,
                             dim_feedforward=2 * dim_hidden,
                             dropout=dropout,
                             num_heads=num_heads,
                             num_layers=num_layers,
                             batch_norm=batch_norm,
                             abs_pe=abs_pe,
                             abs_pe_dim=abs_pe_dim,
                             gnn_type=gnn_type,
                             use_edge_attr=use_edge_attr,
                             num_edge_features=41,
                             edge_dim=edge_dim,
                             k_hop=k_hop,
                             se=se,
                             deg=deg,
                             global_pool=global_pool)

sub_model = sub_model.to(device)
optimizer = optim.AdamW(sub_model.parameters(), lr=lr, weight_decay=weight_decay)
out = train_out(subset_loader, sub_model,optimizer)

loader之后： DataBatch(x=[65, 92], edge_index=[2, 348], edge_attr=[348, 41], y=[10, 1], complete_edge_index=[2, 513], degree=[65], subgraph_edge_index=[2, 2844], num_subgraph_nodes=[10], subgraph_node_idx=[481], subgraph_edge_attr=[2844, 41], subgraph_indicator=[481], batch=[65], ptr=[11])
0
DataBatch(x=[65, 92], edge_index=[2, 348], edge_attr=[348, 41], y=[10, 1], complete_edge_index=[2, 513], subgraph_edge_index=[2, 2844], num_subgraph_nodes=[10], subgraph_node_idx=[481], subgraph_edge_attr=[2844, 41], subgraph_indicator=[481], degree=[65], batch=[65], ptr=[11])


RuntimeError: Sizes of tensors must match except in dimension 2. Expected size 2844 but got size 116604 for tensor number 2 in the list.